In [6]:
import requests
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.google_places.tool import GooglePlacesTool
from langchain_community.utilities.google_places_api import GooglePlacesAPIWrapper
from langchain_community.utilities.serpapi import SerpAPIWrapper
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper

from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState, StateGraph, END, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_groq import ChatGroq
from typing import Dict, List, Any,Literal
import json
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')

In [8]:
class CurrencyService:
    def __init__(self):
        self.api_key = os.getenv("CURRENCY_API_KEY")  # Optional, for future use if needed
        self.base_url = "https://api.exchangerate-api.com/v4/latest"

    def get_exchange_rate(self, from_currency: str, to_currency: str) -> float:
        """Get exchange rate between two currencies"""
        try:
            url = f"{self.base_url}/{from_currency}"
            response = requests.get(url)
            data = response.json()
            if response.status_code == 200 and to_currency in data.get('rates', {}):
                return data['rates'][to_currency]
            return 1.0
        except:
            return 1.0

    def convert_currency(self, amount: float, from_currency: str, to_currency: str) -> float:
        """Convert amount from one currency to another"""
        rate = self.get_exchange_rate(from_currency, to_currency)
        return amount * rate

In [9]:
class TravelBudgetEstimator:
    def __init__(self, total_budget: float, num_days: int = 1, mode: str = "standard"):
        self.total_budget = total_budget
        self.num_days = max(1, num_days)
        self.mode = mode.lower()
        self.ratios = {
            "budget": {"stay": 0.35, "food": 0.30, "transport": 0.20, "activities": 0.15},
            "standard": {"stay": 0.40, "food": 0.30, "transport": 0.20, "activities": 0.10},
            "luxury": {"stay": 0.50, "food": 0.25, "transport": 0.15, "activities": 0.10}
        }

    def estimate_breakdown(self) -> dict:
        ratio = self.ratios.get(self.mode, self.ratios["standard"])
        return {k: round(self.total_budget * v, 2) for k, v in ratio.items()}

    def daily_budget(self) -> float:
        return round(self.total_budget / self.num_days, 2)
class TravelCostCalculator:
    @staticmethod
    def multiply(a: float, b: float) -> float:
        return a * b

    @staticmethod
    def add(a: float, b: float) -> float:
        return a + b

    @staticmethod
    def calculate_total_cost(*costs: float) -> float:
        return sum(costs)

    @staticmethod
    def calculate_daily_budget(total_cost: float, days: int) -> float:
        return total_cost / days if days > 0 else 0

In [10]:
class WeatherService:
    def __init__(self):
        self.api_key = os.getenv("OPENWEATHER_API_KEY")
        self.base_url = "http://api.openweathermap.org/data/2.5"

    def get_current_weather(self, city: str) -> Dict:
        """Get current weather for a city"""
        try:
            url = f"{self.base_url}/weather"
            params = {
                "q": city,
                "appid": os.getenv("WEATHER_API"),
                "units": "metric"
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except Exception as e:
            print(f"Error in get_current_weather: {e}")
            return {}

    def get_weather_forecast(self, city: str, days: int = 5) -> Dict:
        """Get weather forecast for a city (3-hour interval data)"""
        try:
            url = f"{self.base_url}/forecast"
            params = {
                "q": city,
                "appid": os.getenv("WEATHER_API"),
                "units": "metric",
                "cnt": days * 8
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except Exception as e:
            print(f"Error in get_weather_forecast: {e}")
            return {}

In [ ]:
class AgentSetup:
    def __init__(self):
        # DuckDuckGo fallback
        self.search_tool = DuckDuckGoSearchRun()

        # Google Places
        try:
            google_places_key = os.getenv("GOOGLE_PLACES_API_KEY")
            if google_places_key:
                places_wrapper = GooglePlacesAPIWrapper(google_places_api_key=google_places_key)
                self.places_tool = GooglePlacesTool(api_wrapper=places_wrapper)
            else:
                self.places_tool = None
        except Exception:
            self.places_tool = None

        #  SerpAPI
        try:
            serpapi_key = os.getenv("SERPAPI_KEY")
            if serpapi_key:
                self.serp_search = SerpAPIWrapper(serpapi_api_key=serpapi_key)
            else:
                self.serp_search = None
        except Exception:
            self.serp_search = None

        #  Serper
        try:
            serper_key = os.getenv("SERPER_API_KEY")
            if serper_key:
                self.serper_search = GoogleSerperAPIWrapper(serper_api_key=serper_key)
            else:
                self.serper_search = None
        except Exception:
            self.serper_search = None

        # LLM Init
        self.llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

        # Bind Tools
        self.tools = self._setup_tools()
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        self.weather_service = WeatherService()
        self.currency_service = CurrencyService()

    def _setup_tools(self) -> List:
        """Define and bind all tools"""

        @tool
        def search_hotels(city: str,budget_range:str="Below-Expensive") -> str:
            """Search for hotels in a city using real-time tools like Google Places or search engines."""
            query = f"best {budget_range} accomedations hotels to stay in {city} with prices, reviews"

            if self.places_tool:
                try:
                    places_result = self.places_tool.run(f"hotels in {city}")
                    if places_result and len(places_result) > 50:
                        return f"Real-time hotel data: {places_result}"
                except Exception:
                    pass

            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Google hotels data: {serp_result}"
                except Exception:
                    pass

            if self.serper_search:
                try:
                    serper_result = self.serper_search.run(query)
                    if serper_result and len(serper_result) > 50:
                        return f"Latest hotels result: {serper_result}"
                except Exception:
                    pass

            return self.search_tool.invoke(query)

        @tool
        def search_flights(origin: str, destination: str, start_date: str, end_date: str) -> str:
            """
            Search for available round-trip flights between two cities within a date range.
            Args:
                origin: Departure city
                destination: Arrival city
                start_date: Trip start date (format: YYYY-MM-DD)
                end_date: Return date (format: YYYY-MM-DD)
            """
            query = (
                f"flights from {origin} to {destination} "
                f"departing on {start_date} and returning on {end_date} "
                f"with price, airline, and timing info"
            )

            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Flight options via SerpAPI: {serp_result}"
                except Exception:
                    pass

            if self.serper_search:
                try:
                    serper_result = self.serper_search.run(query)
                    if serper_result and len(serper_result) > 50:
                        return f"Flight info via Serper: {serper_result}"
                except Exception:
                    pass

            return self.search_tool.invoke(query)
        @tool
        def get_current_weather(city: str) -> str:
            """
        Get current weather conditions in a city.
            """
            data = self.weather_service.get_current_weather(city)

            if "main" in data:
                temp = data["main"]["temp"]
                feels_like = data["main"]["feels_like"]
                condition = data["weather"][0]["description"]
                wind = data["wind"]["speed"]

                return (
                    f"Current weather in {city}:\n"
                    f"Temperature: {temp}°C (feels like {feels_like}°C)\n"
                    f"Condition: {condition}\n"
                    f"Wind Speed: {wind} m/s"
                )

            return "Weather data is currently unavailable. Try again later or check the city name."
        @tool
        def get_weather_forecast(city: str) -> str:
            """
            Get a 3-day weather forecast for a city.
            """
            forecast_data = self.weather_service.get_weather_forecast(city, days=3)

            if "list" in forecast_data:
                forecast_list = forecast_data["list"][:5]  # First few entries (next 15 hours)
                summary = [f"{f['dt_txt']}: {f['main']['temp']}°C, {f['weather'][0]['description']}" for f in forecast_list]
                return "\n".join(summary)

            return "Weather forecast unavailable. Please try another city or check spelling."

        @tool
        def search_restaurants(city: str) -> str:
            """Search for top-rated restaurants and food places in a city."""
            query = f"best restaurants and food places to eat in {city}"

            # Try Google Places
            if self.places_tool:
                try:
                    places_result = self.places_tool.run(f"restaurants in {city}")
                    if places_result and len(places_result) > 50:
                        return f"Real-time restaurant suggestions: {places_result}"
                except Exception:
                    pass

            # Try SerpAPI
            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Latest from Google search: {serp_result}"
                except Exception:
                    pass

            # Fallback to DuckDuckGo
            return f"🍴 Suggestions via DuckDuckGo:\n{self.search_tool.invoke(query)}"

        @tool
        def search_transportation(city: str) -> str:
            """Search for public and private transportation options in a city."""
            query = f"transportation options in {city} including metro, bus, cab, Uber, and local transit"

            # SerpAPI
            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Local transport options from Google search: {serp_result}"
                except Exception:
                    pass

            # Serper
            if self.serper_search:
                try:
                    serper_result = self.serper_search.run(query)
                    if serper_result and len(serper_result) > 50:
                        return f"Transit information from Serper: {serper_result}"
                except Exception:
                    pass

            # DuckDuckGo fallback
            return f"Transport info from DuckDuckGo:\n{self.search_tool.invoke(query)}"

        @tool
        def search_attractions(city: str) -> str:
            """Search for top tourist attractions and activities in a city using real-time data."""
            query = f"top attractions, activities, and things to do in {city}"

            # Google Places
            if self.places_tool:
                try:
                    places_result = self.places_tool.run(f"tourist attractions in {city}")
                    if places_result and len(places_result) > 50:
                        return f"Real-time attractions data: {places_result}"
                except Exception:
                    pass

            # SerpAPI
            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Latest search results: {serp_result}"
                except Exception:
                    pass

            # Google Serper
            if self.serper_search:
                try:
                    serper_result = self.serper_search.run(query)
                    if serper_result and len(serper_result) > 50:
                        return f"Current search data: {serper_result}"
                except Exception:
                    pass

            # DuckDuckGo fallback
            return f"Attraction results from DuckDuckGo:\n{self.search_tool.invoke(query)}"

        @tool
        def estimate_trip_allocation(total_budget: float, num_days: int = 1, mode: str = "standard") -> str:
            """
            Estimate the budget allocation for a trip.

            Args:
                total_budget (float): Total available budget for the trip.
                num_days (int): Number of days for the trip.
                mode (str): Spending mode — can be "budget", "standard", or "luxury".

            Returns:
                str: A formatted string showing cost breakdown and daily budget.
            """
            estimator = TravelBudgetEstimator(total_budget, num_days, mode)
            breakdown = estimator.estimate_breakdown()
            daily = estimator.daily_budget()
            breakdown_str = "\n".join([f"{k.capitalize()}: ${v}" for k, v in breakdown.items()])
            return f"Estimated Budget Breakdown ({mode}):\n{breakdown_str}\n\nDaily Budget: ${daily}"


        @tool
        def estimate_hotel_cost(price_per_night: float, total_days: int) -> float:
            """
            Estimate the total cost of hotel stay.

            Args:
                price_per_night (float): Cost per night for accommodation.
                total_days (int): Total number of nights staying.

            Returns:
                float: Total hotel cost.
            """
            return TravelCostCalculator.multiply(price_per_night, total_days)


        @tool
        def add_costs(cost1: float, cost2: float) -> float:
            """
            Add two individual cost components together.

            Args:
                cost1 (float): First cost value.
                cost2 (float): Second cost value.

            Returns:
                float: Combined total cost.
            """
            return TravelCostCalculator.add(cost1, cost2)


        @tool
        def calculate_total_expense(*costs: float) -> float:
            """
            Sum multiple cost values to compute the total expense.

            Args:
                *costs (float): Any number of cost components.

            Returns:
                float: Total aggregated cost.
            """
            return TravelCostCalculator.calculate_total_cost(*costs)


        @tool
        def calculate_daily_budget(total_cost: float, days: int) -> float:
            """
            Calculate per-day budget from total cost and number of days.

            Args:
                total_cost (float): Overall expense.
                days (int): Total number of travel days.

            Returns:
                float: Daily cost estimate.
            """
            return TravelCostCalculator.calculate_daily_budget(total_cost, days)
        @tool
        def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
            """Convert currency between two types"""
            return self.currency_service.convert_currency(amount, from_currency, to_currency)

        @tool
        def get_exchange_rate(from_currency: str, to_currency: str) -> float:
            """Get live exchange rate between two currencies"""
            return self.currency_service.get_exchange_rate(from_currency, to_currency)

        @tool
        def create_trip_plan(
            city: str,
            origin: str,
            start_date: str = None,
            end_date: str = None
        ) -> str:
            """
            Create a trip plan with flights, hotel suggestions, and weather forecast.
            If dates are not provided, assumes a 5-day trip.
            """
            from datetime import datetime, timedelta

            try:
                today = datetime.today()

                # Handle start & end date
                if not start_date:
                    start = today + timedelta(days=1)
                    start_date = start.strftime("%Y-%m-%d")
                else:
                    start = datetime.strptime(start_date, "%Y-%m-%d")

                if not end_date:
                    end = start + timedelta(days=4)
                    end_date = end.strftime("%Y-%m-%d")
                else:
                    end = datetime.strptime(end_date, "%Y-%m-%d")

                # Duration
                days = (end - start).days
                if days <= 0:
                    return "Invalid date range. End date must be after start date."

                # Get data from tools
                flights = search_flights(origin, city, start_date,end_date)
                hotel_info = search_hotels(city)
                weather = get_weather_forecast(city, days)

                return (
                    f"📍 Trip Plan for {city} ({start_date} to {end_date})\n"
                    f"Duration: {days} days\n\n"
                    f"✈️ Flights from {origin}:\n{flights}\n\n"
                    f"🏨 Hotel Suggestions:\n{hotel_info[:300]}...\n\n"
                    f"🌦️ Weather Forecast:\n{weather[:300]}...\n"
                )

            except Exception as e:
                return f"Error creating trip plan: {str(e)}"

        @tool
        def create_day_plan(
            city: str,
            day_number: int,
            weather: str,
            attractions: str,
            restaurants: str = None,
            total_budget: float = 0,
            num_days: int = 1,
            mode: str = "standard"
        ) -> str:
            """
            Create a detailed day plan including itinerary and estimated cost for the day.

            Args:
                city (str): City for the trip.
                day_number (int): Which day of the trip.
                weather (str): Forecasted weather info.
                attractions (str): Attractions in the city.
                restaurants (str, optional): Restaurant recommendations.
                total_budget (float, optional): Total trip budget.
                num_days (int, optional): Total number of days for trip.
                mode (str, optional): Travel style - budget / standard / luxury.

            Returns:
                str: Day-wise plan with activities, tips, and cost estimate.
            """
            try:
                # Prepare summaries
                top_attractions = attractions[:300].strip() + "..." if len(attractions) > 300 else attractions
                top_restaurants = restaurants[:300].strip() + "..." if restaurants and len(restaurants) > 300 else restaurants

                # Tips logic
                indoor_suggestion = "Explore museums, art galleries, or cozy cafés."
                outdoor_suggestion = "Plan for parks, walking tours, or scenic outdoor spots."
                tips = indoor_suggestion if "rain" in weather.lower() or "storm" in weather.lower() else outdoor_suggestion

                # Budget estimation
                estimator = TravelBudgetEstimator(total_budget, num_days, mode)
                breakdown = estimator.estimate_breakdown()
                daily = estimator.daily_budget()

                breakdown_str = "\n".join([f"  - {k.capitalize()}: ${v}" for k, v in breakdown.items()])
                cost_block = f"""
        💰 Estimated Cost for Day {day_number} ({mode.title()} Trip): ${daily}

        Breakdown:
        {breakdown_str}
        """.strip()

                # Final plan output
                plan = f"""
        🗓️ Day {day_number} in {city}
        🌤️ Weather: {weather}

        📍 Morning:
        - Start your day with light breakfast nearby.
        - Visit: {top_attractions.split(',')[0].strip()}.

        📍 Afternoon:
        - Continue exploring: {top_attractions.split(',')[1].strip() if ',' in top_attractions else top_attractions}.
        - Lunch break at a local spot.

        📍 Evening:
        - Enjoy the city vibes or sunset view.
        - Dinner at: {top_restaurants if top_restaurants else 'a recommended local restaurant'}.

        💡 Tips: {tips}

        {cost_block}
        """.strip()

                return plan

            except Exception as e:
                return f"Error generating day plan: {str(e)}"

        return [
    search_hotels,
    search_attractions,
    search_flights,
    search_restaurants,
    search_transportation,
    get_current_weather,
    get_weather_forecast,
    estimate_trip_allocation,
    estimate_hotel_cost,
    add_costs,
    calculate_total_expense,
    calculate_daily_budget,
    convert_currency,
    get_exchange_rate,
    create_trip_plan,
    create_day_plan,
]


In [12]:
agent = AgentSetup()

response = agent.llm_with_tools.invoke("Find me best hotels in Jaipur and cheap flights from Delhi to Jaipur")

In [13]:
response.tool_calls

[{'name': 'search_hotels',
  'args': {'budget_range': 'Below-Expensive', 'city': 'Jaipur'},
  'id': '1c19zfzwn',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.messages import BaseMessage, SystemMessage
from langchain_core.messages import HumanMessage
MessagesState = dict[str, list[BaseMessage]]
from typing import Literal

class TravelAgent:
    def __init__(self, travel_planner):
        self.travel_planner = travel_planner
        self.system_prompt = self._build_system_prompt()
        self.graph = self._build_graph()

    def _build_system_prompt(self) -> SystemMessage:
        return SystemMessage(
            content=(
                "You are a helpful AI Travel Agent and Expense Planner.\n"
                "You help users plan trips to any city worldwide with real-time data.\n\n"
                "IMPORTANT: Always provide COMPLETE and DETAILED travel plans.\n"
                "Never say 'I'll prepare' or 'hold on'. Provide full information immediately:\n"
                "- Complete day-by-day itinerary\n"
                "- Specific attractions with details\n"
                "- Restaurant recommendations with prices\n"
                "- Detailed cost breakdown\n"
                "- Transportation information\n"
                "- Weather details\n\n"
                "Use available tools to gather real-time information and make accurate calculations.\n"
                "Respond in clean Markdown format."
            )
        )

    def _agent_node(self, state: MessagesState) -> MessagesState:
        """Invoke the LLM with system prompt + chat history."""
        messages = [self.system_prompt] + state["messages"]
        response = self.travel_planner.llm_with_tools.invoke(messages)
        return {"messages": [response]}

    def _should_continue(self, state: MessagesState) -> Literal["tools", "__end__"]:
        """Check whether to continue tool calls or finish."""
        last_message = state["messages"][-1]
        content = last_message.content.lower()

        if getattr(last_message, "tool_calls", None):
            return "tools"

        if any(p in content for p in [
            "let me search", "i'll look up", "please hold on",
            "i'll prepare", "let me gather", "i need to check"
        ]):
            return "tools"

        if len(content) < 500:
            return "tools"

        keywords = ["hotel", "attraction", "cost", "weather", "itinerary"]
        if sum(1 for k in keywords if k in content) < 3:
            return "tools"

        return "__end__"

    def _build_graph(self):
        """Builds and compiles the LangGraph agent workflow."""
        graph = StateGraph(MessagesState)
        graph.add_node("agent", self._agent_node)
        graph.add_node("tools", ToolNode(self.travel_planner.tools))

        graph.add_edge(START, "agent")
        graph.add_conditional_edges("agent", self._should_continue)
        graph.add_edge("tools", "agent")

        return graph.compile()

    def plan_trip(self, user_input: str, max_iterations: int = 10) -> str:
        """
        Main entrypoint to plan a trip using LangGraph agent flow.

        - Executes graph with the user query.
        - Controls loop with max_iterations (recursion limit).
        - Triggers a summarization prompt if LLM response is short.
        - Falls back to a direct LLM response if toolchain fails.
        """
        messages = [HumanMessage(content=user_input)]
        config = {"recursion_limit": max_iterations}
        response_messages = messages  # fallback default
        final_response = ""

        try:
            # Stream the graph execution step-by-step
            stream = self.graph.stream({"messages": messages}, config=config)

            for step in stream:
                print("\n🔄 LangGraph Step Output")
                print("=" * 50)

                for key, value in step.items():
                    print(f"🧩 Key: {key}")
                    print("📤 Value:")

                    # If this is the main message stream
                    if key == "messages" and isinstance(value, list):
                        for msg in value:
                            try:
                                msg_type = getattr(msg, "type", "Unknown").capitalize()
                                content = getattr(msg, "content", "")
                                print(f"[{msg_type}] {content[:300]}...\n")
                            except Exception as e:
                                print(f"⚠️ Error printing message: {e}")
                        # Update the main message chain
                        response_messages = value
                    else:
                        print(value)

                    print("-" * 50)

            # Extract final response if any
            if response_messages:
                final_response = response_messages[-1].content

            # Fallback summarization if response seems short
            if len(final_response) < 800:
                summary_prompt = (
                    f"Generate a complete final summary based on the current planning data.\n\n"
                    f"Do NOT call any tools again. Just use the information you've already gathered.\n\n"
                    "### Respond in well-formatted Markdown, covering:\n"
                    "- ✅ Full day-by-day itinerary\n"
                    "- 📍 Top attractions\n"
                    "- 🍴 Restaurant suggestions\n"
                    "- 💸 Cost estimates\n"
                    "- 🌦️ Weather forecast summary\n"
                    "- 🚕 Transportation options\n\n"
                    f"Original user request: {user_input}"
                )
                summary_messages = response_messages + [HumanMessage(content=summary_prompt)]
                summary_response = self.travel_planner.llm_with_tools.invoke(summary_messages)
                return summary_response.content

            return final_response or "⚠️ No final response generated."

        except Exception as e:
            print(f"[TravelAgent] Graph execution error: {e}")
            return self._fallback_planning(user_input)

    def _fallback_planning(self, user_input: str) -> str:
        """
        Fallback if the graph fails or tool invocation breaks.
        Uses a direct LLM prompt to produce a complete travel plan without tools.
        """
        fallback_prompt = (
            f"Your task is to create a complete travel plan for the following user request:\n\n"
            f"🧳 **Trip Request**: {user_input}\n\n"
            "Build a comprehensive and realistic travel itinerary. Assume real-world knowledge, but do not use tools or real-time data.\n\n"
            "### Please include the following sections:\n"
            "1. **Day-by-Day Itinerary** – Include what to do each day, with times of day and key experiences.\n"
            "2. **Top Attractions** – Highlight must-see places and experiences.\n"
            "3. **Restaurant Suggestions** – Recommend food spots with cuisine types and approximate cost.\n"
            "4. **Estimated Budget** – Breakdown total cost into stay, food, activities, transport.\n"
            "5. **Weather Overview** – Describe the expected weather during travel (assume typical season).\n"
            "6. **Transportation Guide** – Explain how to get around the city (public, walking, ride apps).\n\n"
            "**Formatting instructions:**\n"
            "- Use clean Markdown with proper headers (##, ###, etc)\n"
            "- Use bullet points or lists where needed\n"
            "- Keep tone friendly but informative\n"
            "- DO NOT mention that tools were not used\n"
            "- DO NOT say 'I assume' or 'I think'\n"
        )

        messages = [self.system_prompt, HumanMessage(content=fallback_prompt)]
        response = self.travel_planner.llm_with_tools.invoke(messages)
        return response.content

    def export_to_markdown(self, response_text: str, filename: str = "ai_travel_plan.md") -> str:
        """
        Export travel plan to a well-formatted Markdown file.
        Includes metadata, disclaimer, and timestamp.
        """
        from datetime import datetime

        # 🧠 Build Markdown content
        markdown_content = f""" AI Travel Itinerary

    **Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    **Planner:** TravelAgent by Arun 🚀

    ---

    {response_text}

    ---

    > ℹ️ *Note: This travel plan was AI-generated. Please verify all costs, places, and local regulations before booking.*
    """

        try:
            with open(filename, "w", encoding="utf-8") as f:
                f.write(markdown_content)

            print(f"[Export] Markdown file saved: {filename}")
            return filename

        except Exception as e:
            print(f"[Export Error] Could not save file: {e}")
            return None

In [24]:
def main():
    # Step 1: Initialize agent and travel planner
    agent = AgentSetup()
    travel_assistant = TravelAgent(travel_planner=agent)

    # Step 2: User query (example)
    user_query = (
        "Give me a detailed 6 Day trip itinerary to london including best view hotels and Flight Tickets Up and Down , "
        "historic places, food recommendations, all within a 1 lacks  rupees budget and convert that to indian exchange ."
    )

    print("AI Travel Agent - Trip Planning")
    print("=" * 50)
    print(f"Request: {user_query}")
    print("Generating travel plan...\n")

    try:
        # Step 3: Generate the travel plan
        response = travel_assistant.plan_trip(user_query, max_iterations=10)

        # Step 4: Save to Markdown
        filename = f"travel_plan_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
        saved_file = travel_assistant.export_to_markdown(response, filename)

        if saved_file:
            print("Travel plan generated successfully.")
            print(f"Saved as: {saved_file}")
            print(f"Response length: {len(response)} characters")
        else:
            print("Failed to save Markdown file.")

    except Exception as e:
        print(f"Error during travel planning: {str(e)}")

    print("=" * 50)


if __name__ == "__main__":
    main()


AI Travel Agent - Trip Planning
Request: Give me a detailed 6 Day trip itinerary to london including best view hotels and Flight Tickets Up and Down , historic places, food recommendations, all within a 1 lacks  rupees budget and convert that to indian exchange .
Generating travel plan...


🔄 LangGraph Step Output
🧩 Key: agent
📤 Value:
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'tfk4egxbm', 'function': {'arguments': '{"mode":"standard","num_days":6,"total_budget":100000}', 'name': 'estimate_trip_allocation'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 527, 'prompt_tokens': 1532, 'total_tokens': 2059, 'completion_time': 2.357308788, 'prompt_time': 0.108691837, 'queue_time': 0.28947413299999997, 'total_time': 2.466000625}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_87ec4d1ec3', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--75d56cef-7e84-4065-b46c-677a4539d5b6-0', tool_calls=[{